In [3]:
# docker run --name mysql_cnt -e MYSQL_ROOT_PASSWORD=codeit -d -p 3307:3306 mysql

from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [5]:
import os

for i in os.listdir('dataset'):
    print(i)
    print(pd.read_csv(f"dataset/{i}").shape)

orders.csv
(47079, 5)
pizza_types.csv
(32, 4)
pizzas.csv
(96, 4)


In [20]:
engine_name = 'mysql+pymysql'
user_id = 'root'
user_pw = 'codeit'
host = 'localhost'
port = '3307'
database = 'mysql'

db = create_engine(f'{engine_name}://{user_id}:{user_pw}@{host}:{port}/{database}?charset=utf8')

cursor = db.connect()
Session = sessionmaker(db)
session = Session()

In [21]:
q_ = "CREATE DATABASE codeit;"

cursor.execute(text(q_))

Exception during reset or similar
Traceback (most recent call last):
  File "/Users/hyunsoo/Desktop/Lecture/lecture/lib/python3.12/site-packages/sqlalchemy/engine/base.py", line 1910, in _execute_context
    self.dialect.do_execute(
  File "/Users/hyunsoo/Desktop/Lecture/lecture/lib/python3.12/site-packages/sqlalchemy/engine/default.py", line 736, in do_execute
    cursor.execute(statement, parameters)
  File "/Users/hyunsoo/Desktop/Lecture/lecture/lib/python3.12/site-packages/pymysql/cursors.py", line 153, in execute
    result = self._query(query)
             ^^^^^^^^^^^^^^^^^^
  File "/Users/hyunsoo/Desktop/Lecture/lecture/lib/python3.12/site-packages/pymysql/cursors.py", line 322, in _query
    conn.query(q)
  File "/Users/hyunsoo/Desktop/Lecture/lecture/lib/python3.12/site-packages/pymysql/connections.py", line 563, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Us

ProgrammingError: (pymysql.err.ProgrammingError) (1007, "Can't create database 'codeit'; database exists")
[SQL: CREATE DATABASE codeit;]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [22]:
query = "show databases"

pd.read_sql(sql=query, con=db)

,Database
0,codeit
1,information_schema
2,mysql
3,performance_schema
4,sys


In [23]:
## orders
orders_create = """
CREATE TABLE IF NOT EXISTS codeit.orders (
    order_id INT,
    order_timestamp DATETIME,
    order_details_id INT,
    pizza_id VARCHAR(50),
    quantity INT,
    PRIMARY KEY (order_id, order_details_id)
);
"""

## pizza_types
pizza_types_create = """
CREATE TABLE IF NOT EXISTS codeit.pizza_types (
    pizza_type_id VARCHAR(50),
    name VARCHAR(100),
    category VARCHAR(50),
    ingredients TEXT
);
"""

## pizzas
pizzas_create = """
CREATE TABLE IF NOT EXISTS codeit.pizzas (
    pizza_id VARCHAR(50),
    pizza_type_id VARCHAR(100),
    size CHAR(3),
    price INT,
    PRIMARY KEY (pizza_id)
);
"""

아래는 제공된 테이블을 기반으로 한 컬럼 정의서입니다:

#### 1. `orders`
- **order_id** : 주문을 식별하는 고유 ID. (`INT`)
- **order_timestamp** : 주문이 발생한 날짜와 시간. (`DATETIME`)
- **order_details_id** : 주문 내 세부 항목을 식별하는 고유 ID. (`INT`)
- **pizza_id** : 주문된 피자를 식별하는 고유 ID. (`VARCHAR`)
- **quantity** : 해당 피자의 주문 수량. (`INT`)

#### 2. `pizza_types`
- **pizza_type_id** : 피자 종류를 식별하는 고유 ID. (`VARCHAR`)
- **name** : 피자의 이름. (`VARCHAR`)
- **category** : 피자의 카테고리 또는 유형 (예: Classic, Chicken 등). (`VARCHAR`)
- **ingredients** : 피자에 포함된 재료 목록. (`TEXT`)

#### 3. `pizzas`
- **pizza_id** : 피자의 고유 ID. (`VARCHAR`)
- **pizza_type_id** : 피자 종류를 식별하는 ID로, `pizza_types` 테이블의 `pizza_type_id`와 연관. (`VARCHAR`)
- **size** : 피자의 크기 (예: L, M, S). (`CHAR(1)`)
- **price** : 피자의 가격. (`INT`)

In [24]:
create_query = [orders_create, pizza_types_create, pizzas_create]

for q_ in create_query:
    cursor = db.connect()
    cursor.execute(text(q_))

In [25]:
query = "show tables from codeit"

pd.read_sql(sql=query, con=db)

,Tables_in_codeit
0,orders
1,pizza_types
2,pizzas


In [16]:
cursor = db.connect()
cursor.execute(text("drop table codeit.orders"))

AttributeError: 'Connection' object has no attribute 'commit'

In [26]:
ord = pd.read_csv('./dataset/orders.csv')
ord.to_sql(name='orders', con=db, if_exists='append', index=False)

47079

In [27]:
pt = pd.read_csv('./dataset/pizza_types.csv')
pt.to_sql(name='pizza_types', con=db, if_exists='append', index=False)

32

In [28]:
pizza = pd.read_csv('./dataset/pizzas.csv')
pizza.to_sql(name='pizzas', con=db, if_exists='append', index=False)

96

- unique 영업일 수
- 총 매출
- 하루 평균 매출
- 1년동안 주문 횟수
- 평균 주문 금액
- 총 판매량
- 한 주문당 시킨 피자 판 수 내림차순 정렬
---
- 월별 매출
- 요일별 매출
- 시간대별 매출
---
- 피자 메뉴별 판매량
- 각 피자 메뉴의 월별 판매량
- 각 피자 메뉴의 사이즈별 판매량
- 피자 카테고리별 판매량
- ingredients 사용량

In [29]:
df_query = ["orders", "pizza_types", "pizzas"]

for tbl in df_query:
    q_ = f"SELECT * FROM {tbl} LIMIT 10"
    print(tbl)
    display(pd.read_sql(q_, db))

orders


,order_id,order_timestamp,order_details_id,pizza_id,quantity
0,1,2023-01-01 11:45:11,1,hawaiian_m,1
1,2,2023-01-01 11:53:26,2,big_meat_l,1
2,2,2023-01-01 11:53:26,3,classic_dlx_m,1
3,2,2023-01-01 11:53:26,4,five_cheese_l,1
4,2,2023-01-01 11:53:26,5,potato_m,1
5,2,2023-01-01 11:53:26,6,thai_ckn_l,1
6,3,2023-01-01 11:54:22,7,pepperoni_l,1
7,3,2023-01-01 11:54:22,8,super_supreme_m,1
8,4,2023-01-01 12:09:18,9,big_meat_m,1
9,5,2023-01-01 12:15:29,10,super_supreme_m,1


pizza_types


,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."
5,thai_ckn,The Thai Chicken Pizza,Chicken,"Chicken, Pineapple, Tomatoes, Red Peppers, Tha..."
6,big_meat,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sau..."
7,classic_dlx,The Classic Deluxe Pizza,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,..."
8,hawaiian,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
9,ital_cpcllo,The Italian Capocollo Pizza,Classic,"Capocollo, Red Peppers, Tomatoes, Goat Cheese,..."


pizzas


,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,20400
1,bbq_ckn_m,bbq_ckn,M,26800
2,bbq_ckn_l,bbq_ckn,L,33200
3,cali_ckn_s,cali_ckn,S,20400
4,cali_ckn_m,cali_ckn,M,26800
5,cali_ckn_l,cali_ckn,L,33200
6,ckn_alfredo_s,ckn_alfredo,S,20400
7,ckn_alfredo_m,ckn_alfredo,M,26800
8,ckn_alfredo_l,ckn_alfredo,L,33200
9,ckn_pesto_s,ckn_pesto,S,20400


In [ ]:
query = "SELECT * FROM pizza_types WHERE ingredients LIKE '%%Tomatoes%%'"

pd.read_sql(sql=query, con=db)

In [44]:
q_ = """
SELECT DATE_FORMAT(order_timestamp, '%%Y-%%m') AS month, SUM(p.price * o.quantity) AS monthly_revenue
FROM orders o
JOIN pizzas p ON o.pizza_id = p.pizza_id
GROUP BY month
ORDER BY month DESC;
"""


pd.read_sql(sql=q_, con=db)

,month,monthly_revenue
0,2023-12,83628400.0
1,2023-11,84797600.0
2,2023-10,76953200.0
3,2023-09,78980000.0
4,2023-08,82976000.0
5,2023-07,88316800.0
6,2023-06,81142800.0
7,2023-05,87732400.0
8,2023-04,82263600.0
9,2023-03,84837200.0


In [50]:
q_ = """
SELECT pt.name AS pizza_name, p.size, SUM(o.quantity) AS total_quantity_sold
FROM orders o
JOIN pizzas p ON o.pizza_id = p.pizza_id
JOIN pizza_types pt ON p.pizza_type_id = pt.pizza_type_id
GROUP BY pt.name, p.size
ORDER BY pt.name, p.size;
"""


pd.read_sql(sql=q_, con=db)

,pizza_name,size,total_quantity_sold
0,The Barbecue Chicken Pizza,L,1674.0
1,The Barbecue Chicken Pizza,M,1528.0
2,The Barbecue Chicken Pizza,S,863.0
3,The Big Meat Pizza,L,905.0
4,The Big Meat Pizza,M,769.0
5,The Big Meat Pizza,S,2471.0
6,The Classic Deluxe Pizza,L,1393.0
7,The Classic Deluxe Pizza,M,1959.0
8,The Classic Deluxe Pizza,S,1339.0
9,The Five Cheese Pizza,L,2299.0


In [38]:
q_ = """
SELECT DISTINCT YEAR(order_timestamp) AS total_orders
FROM orders
"""


pd.read_sql(sql=q_, con=db)

,total_orders
0,2023


In [5]:
query = """
SELECT od.*, p.* FROM orders od 
LEFT JOIN pizzas p ON od.pizza_id = p.pizza_id
"""

pd.read_sql(sql=query, con=db)

,order_id,order_timestamp,order_details_id,pizza_id,quantity,pizza_id,pizza_name,size,price
0,1,2023-01-01 11:45:11,1,hawaiian_m,1,hawaiian_m,하와이안 피자,M,21800
1,2,2023-01-01 11:53:26,2,big_meat_l,1,big_meat_l,빅 미트 피자,L,34000
2,2,2023-01-01 11:53:26,3,classic_dlx_m,1,classic_dlx_m,클래식 디럭스 피자,M,20300
3,2,2023-01-01 11:53:26,4,five_cheese_l,1,five_cheese_l,파이브 치즈 피자,L,31000
4,2,2023-01-01 11:53:26,5,potato_m,1,potato_m,포테이토 피자,M,21800
...,...,...,...,...,...,...,...,...,...
47074,21348,2023-12-31 20:37:13,47075,bulgogi_m,1,bulgogi_m,불고기 피자,M,20300
47075,21348,2023-12-31 20:37:13,47076,potato_l,1,potato_l,포테이토 피자,L,33800
47076,21348,2023-12-31 20:37:13,47077,super_supreme_s,1,super_supreme_s,슈퍼 슈프림 피자,S,10800
47077,21349,2023-12-31 21:02:45,47078,bulgogi_l,1,bulgogi_l,불고기 피자,L,32300
